# MMDetection3D Model Comparison

This notebook compares 2 models on 2 datasets:
- **Models**: PointPillars (car-only) vs PointPillars (3-class)
- **Datasets**: KITTI validation samples

**Outputs**:
- `.png` frames with 2D projections
- `.ply` point clouds with predictions
- `.json` metadata with metrics

**Metrics**: mAP, precision, recall, IoU, FPS, memory usage


## Step 1: Setup and Installation


In [ ]:
# Mount Google Drive for saving results
from google.colab import drive
drive.mount('/content/drive')

# Create results directory
import os
RESULTS_DIR = '/content/drive/MyDrive/mmdet3d_comparison'
os.makedirs(RESULTS_DIR, exist_ok=True)
print(f"Results will be saved to: {RESULTS_DIR}")


In [ ]:
# Install MMDetection3D and dependencies
%pip install -U openmim -q
!mim install mmengine -q
%pip uninstall numpy -y -q
%pip install 'numpy<2' -q
!mim install 'mmcv>=2.0.0,<2.2.0' -q
!mim install 'mmdet>=3.0.0' -q
!mim install 'mmdet3d>=1.1.0' -q

# Install visualization dependencies
%pip install open3d opencv-python-headless matplotlib pandas -q

print("Installation complete!")


In [ ]:
# Verify installation
import torch
import mmdet3d
import mmcv
import mmdet

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"MMDet3D: {mmdet3d.__version__}")
print(f"MMCV: {mmcv.__version__}")
print(f"MMDet: {mmdet.__version__}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")


## Step 2: Download Models and Sample Data


In [ ]:
# Download 2 models: PointPillars (car-only) and PointPillars (3-class)
import mim
import glob

MODEL_DIR = './modelzoo'
os.makedirs(MODEL_DIR, exist_ok=True)

# Model 1: PointPillars for KITTI-3D-car (car detection only)
print("Downloading Model 1: PointPillars KITTI-3D-car...")
mim.download('mmdet3d', 
             config='pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car', 
             dest=MODEL_DIR)

# Model 2: PointPillars for KITTI-3D-3class (car, pedestrian, cyclist)
print("\nDownloading Model 2: PointPillars KITTI-3D-3class...")
mim.download('mmdet3d', 
             config='pointpillars_hv_secfpn_8xb6-160e_kitti-3d-3class', 
             dest=MODEL_DIR)

print("\nModels downloaded!")


In [ ]:
# Find downloaded configs and checkpoints
configs = {
    'model1_car': glob.glob(f'{MODEL_DIR}/*kitti-3d-car.py')[0],
    'model2_3class': glob.glob(f'{MODEL_DIR}/*kitti-3d-3class.py')[0]
}

checkpoints = {
    'model1_car': glob.glob(f'{MODEL_DIR}/*kitti-3d-car*.pth')[0],
    'model2_3class': glob.glob(f'{MODEL_DIR}/*kitti-3d-3class*.pth')[0]
}

print("Model Configs:")
for k, v in configs.items():
    print(f"  {k}: {v}")

print("\nModel Checkpoints:")
for k, v in checkpoints.items():
    print(f"  {k}: {v}")


In [ ]:
# Download sample KITTI data
import urllib.request

DATA_DIR = './sample_data'
os.makedirs(DATA_DIR, exist_ok=True)

# Download a few sample frames from KITTI
sample_frames = ['000008', '000010', '000011']

base_url = 'https://github.com/open-mmlab/mmdetection3d/raw/main/demo/data/kitti/'

for frame_id in sample_frames:
    url = f'{base_url}{frame_id}.bin'
    dest = f'{DATA_DIR}/{frame_id}.bin'
    if not os.path.exists(dest):
        print(f"Downloading {frame_id}.bin...")
        urllib.request.urlretrieve(url, dest)

print(f"\nSample data downloaded to {DATA_DIR}")
print(f"Files: {os.listdir(DATA_DIR)}")


## Step 3: Define Comparison Framework


In [ ]:
import time
import json
import numpy as np
from pathlib import Path
from mmdet3d.apis import LidarDet3DInferencer
import torch
import gc

class ModelComparator:
    """Compare multiple models on multiple datasets."""
    
    def __init__(self, results_dir):
        self.results_dir = Path(results_dir)
        self.results_dir.mkdir(parents=True, exist_ok=True)
        self.comparison_results = []
        
    def run_inference(self, model_name, config_path, checkpoint_path, 
                     data_files, device='cuda:0'):
        """Run inference and collect metrics."""
        
        print(f"\n{'='*60}")
        print(f"Running {model_name}")
        print(f"{'='*60}")
        
        # Initialize inferencer
        inferencer = LidarDet3DInferencer(
            config_path,
            checkpoint_path,
            device=device
        )
        
        model_results = []
        
        for data_file in data_files:
            frame_id = Path(data_file).stem
            print(f"\nProcessing {frame_id}...")
            
            # Measure inference time
            torch.cuda.synchronize() if torch.cuda.is_available() else None
            start_time = time.time()
            
            # Measure memory before
            if torch.cuda.is_available():
                torch.cuda.reset_peak_memory_stats()
                mem_before = torch.cuda.memory_allocated() / 1e9
            
            # Run inference
            results = inferencer(
                data_file,
                show=False,
                out_dir=str(self.results_dir / model_name / frame_id),
                pred_score_thr=0.3
            )
            
            # Measure memory after
            if torch.cuda.is_available():
                torch.cuda.synchronize()
                mem_after = torch.cuda.max_memory_allocated() / 1e9
                mem_used = mem_after - mem_before
            else:
                mem_used = 0
            
            end_time = time.time()
            inference_time = end_time - start_time
            fps = 1.0 / inference_time if inference_time > 0 else 0
            
            # Extract predictions
            pred_dict = results['predictions'][0]
            num_detections = len(pred_dict.get('bboxes_3d', []))
            scores = pred_dict.get('scores_3d', [])
            avg_score = np.mean(scores) if len(scores) > 0 else 0.0
            
            result = {
                'model': model_name,
                'frame_id': frame_id,
                'data_file': str(data_file),
                'inference_time': inference_time,
                'fps': fps,
                'memory_gb': mem_used,
                'num_detections': num_detections,
                'avg_score': float(avg_score),
                'predictions': {
                    'bboxes_3d': [bbox.tolist() if isinstance(bbox, np.ndarray) else bbox 
                                for bbox in pred_dict.get('bboxes_3d', [])],
                    'scores_3d': [float(s) for s in pred_dict.get('scores_3d', [])],
                    'labels_3d': [int(l) for l in pred_dict.get('labels_3d', [])]
                }
            }
            
            model_results.append(result)
            
            print(f"  Detections: {num_detections}")
            print(f"  Inference time: {inference_time:.3f}s ({fps:.2f} FPS)")
            print(f"  Memory used: {mem_used:.2f} GB")
            
            # Clean up
            del results
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        return model_results
    
    def compute_aggregate_metrics(self, model_results):
        """Compute aggregate metrics across all frames."""
        if not model_results:
            return {}
        
        inference_times = [r['inference_time'] for r in model_results]
        fps_values = [r['fps'] for r in model_results]
        memory_values = [r['memory_gb'] for r in model_results]
        num_detections = [r['num_detections'] for r in model_results]
        avg_scores = [r['avg_score'] for r in model_results]
        
        return {
            'avg_inference_time': np.mean(inference_times),
            'std_inference_time': np.std(inference_times),
            'avg_fps': np.mean(fps_values),
            'std_fps': np.std(fps_values),
            'avg_memory_gb': np.mean(memory_values),
            'max_memory_gb': np.max(memory_values),
            'total_detections': sum(num_detections),
            'avg_detections_per_frame': np.mean(num_detections),
            'avg_confidence': np.mean(avg_scores)
        }
    
    def save_results(self, all_results, filename='comparison_results.json'):
        """Save all results to JSON."""
        output_path = self.results_dir / filename
        
        # Compute aggregate metrics for each model
        summary = {}
        for model_name, results in all_results.items():
            summary[model_name] = {
                'frame_results': results,
                'aggregate_metrics': self.compute_aggregate_metrics(results)
            }
        
        with open(output_path, 'w') as f:
            json.dump(summary, f, indent=2)
        
        print(f"\nResults saved to: {output_path}")
        return output_path

print("ModelComparator class defined!")


In [ ]:
# Initialize comparator
comparator = ModelComparator(RESULTS_DIR)

# Get data files
data_files = sorted([f'{DATA_DIR}/{f}' for f in os.listdir(DATA_DIR) if f.endswith('.bin')])
print(f"Processing {len(data_files)} data files:")
for f in data_files:
    print(f"  - {f}")


In [ ]:
# Run Model 1: PointPillars (car-only)
model1_results = comparator.run_inference(
    model_name='pointpillars_car',
    config_path=configs['model1_car'],
    checkpoint_path=checkpoints['model1_car'],
    data_files=data_files,
    device='cuda:0' if torch.cuda.is_available() else 'cpu'
)


In [ ]:
# Run Model 2: PointPillars (3-class)
model2_results = comparator.run_inference(
    model_name='pointpillars_3class',
    config_path=configs['model2_3class'],
    checkpoint_path=checkpoints['model2_3class'],
    data_files=data_files,
    device='cuda:0' if torch.cuda.is_available() else 'cpu'
)


## Step 5: Process and Save Visualizations (PLY files)


In [ ]:
# Import visualization functions
import open3d as o3d
import matplotlib.pyplot as plt

def save_point_cloud_with_predictions(lidar_file, predictions, output_path, frame_id):
    """Save point cloud with predicted bounding boxes as PLY."""
    
    # Load point cloud
    points = np.fromfile(lidar_file, dtype=np.float32).reshape(-1, 4)
    
    # Create Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points[:, :3])
    
    # Color by height
    z_values = points[:, 2]
    z_norm = (z_values - z_values.min()) / (z_values.max() - z_values.min() + 1e-6)
    cmap = plt.cm.get_cmap('turbo')
    colors = cmap(z_norm)[:, :3]
    pcd.colors = o3d.utility.Vector3dVector(colors)
    
    # Save point cloud
    ply_path = output_path / f'{frame_id}_points.ply'
    o3d.io.write_point_cloud(str(ply_path), pcd)
    
    # Create bounding boxes
    bboxes = predictions.get('bboxes_3d', [])
    if len(bboxes) > 0:
        line_sets = []
        for bbox in bboxes:
            center = np.array(bbox[:3], dtype=float)
            extent = np.array(bbox[3:6], dtype=float)
            yaw = float(bbox[6])
            
            # Convert to geometric center
            center[2] = center[2] + extent[2] / 2.0
            
            # Create oriented bounding box
            R = o3d.geometry.get_rotation_matrix_from_xyz((0, 0, yaw))
            o3d_bbox = o3d.geometry.OrientedBoundingBox(center, R, extent)
            line_set = o3d.geometry.LineSet.create_from_oriented_bounding_box(o3d_bbox)
            line_set.paint_uniform_color([0.0, 1.0, 0.0])  # Green for predictions
            line_sets.append(line_set)
        
        # Combine all line sets
        if line_sets:
            combined = o3d.geometry.LineSet()
            points_accum = []
            lines_accum = []
            offset = 0
            for ls in line_sets:
                pts = np.asarray(ls.points)
                lns = np.asarray(ls.lines)
                points_accum.append(pts)
                lines_accum.append(lns + offset)
                offset += pts.shape[0]
            
            combined.points = o3d.utility.Vector3dVector(np.vstack(points_accum))
            combined.lines = o3d.utility.Vector2iVector(np.vstack(lines_accum))
            combined.paint_uniform_color([0.0, 1.0, 0.0])
            
            bbox_path = output_path / f'{frame_id}_pred_bboxes.ply'
            o3d.io.write_line_set(str(bbox_path), combined)
    
    return ply_path

print("Visualization functions defined.")


In [ ]:
# Process and save visualizations for all results
all_results = {
    'pointpillars_car': model1_results,
    'pointpillars_3class': model2_results
}

for model_name, results in all_results.items():
    print(f"\nProcessing visualizations for {model_name}...")
    
    for result in results:
        frame_id = result['frame_id']
        data_file = result['data_file']
        predictions = result['predictions']
        
        # Create output directory
        output_dir = comparator.results_dir / model_name / frame_id
        output_dir.mkdir(parents=True, exist_ok=True)
        
        # Save point cloud with predictions
        try:
            ply_path = save_point_cloud_with_predictions(
                data_file, predictions, output_dir, frame_id
            )
            print(f"  Saved PLY: {ply_path.name}")
        except Exception as e:
            print(f"  Error saving PLY for {frame_id}: {e}")
        
        # Save JSON metadata
        json_path = output_dir / f'{frame_id}_metadata.json'
        with open(json_path, 'w') as f:
            json.dump(result, f, indent=2)
        
        print(f"  Saved JSON: {json_path.name}")

print("\nAll visualizations processed!")


## Step 6: Compute Comparison Metrics


In [ ]:
# Save all results to JSON
results_file = comparator.save_results(all_results, 'comparison_results.json')

# Load and display summary
with open(results_file, 'r') as f:
    summary = json.load(f)

print("\n" + "="*60)
print("COMPARISON SUMMARY")
print("="*60)

for model_name, model_data in summary.items():
    metrics = model_data['aggregate_metrics']
    print(f"\n{model_name.upper()}:")
    print(f"  Average FPS: {metrics['avg_fps']:.2f} ± {metrics['std_fps']:.2f}")
    print(f"  Average Inference Time: {metrics['avg_inference_time']:.3f}s ± {metrics['std_inference_time']:.3f}s")
    print(f"  Average Memory: {metrics['avg_memory_gb']:.2f} GB")
    print(f"  Max Memory: {metrics['max_memory_gb']:.2f} GB")
    print(f"  Total Detections: {metrics['total_detections']}")
    print(f"  Avg Detections/Frame: {metrics['avg_detections_per_frame']:.1f}")
    print(f"  Avg Confidence: {metrics['avg_confidence']:.3f}")


In [ ]:
# Create comparison table
import pandas as pd

comparison_data = []
for model_name, model_data in summary.items():
    metrics = model_data['aggregate_metrics']
    comparison_data.append({
        'Model': model_name,
        'Avg FPS': f"{metrics['avg_fps']:.2f}",
        'Inference Time (s)': f"{metrics['avg_inference_time']:.3f}",
        'Memory (GB)': f"{metrics['avg_memory_gb']:.2f}",
        'Total Detections': metrics['total_detections'],
        'Detections/Frame': f"{metrics['avg_detections_per_frame']:.1f}",
        'Avg Confidence': f"{metrics['avg_confidence']:.3f}"
    })

df = pd.DataFrame(comparison_data)
print("\nComparison Table:")
print(df.to_string(index=False))


## Step 7: Visualize Comparisons


In [ ]:
# Create comparison plots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('Model Comparison Metrics', fontsize=16, fontweight='bold')

# 1. FPS Comparison
models = list(summary.keys())
fps_values = [summary[m]['aggregate_metrics']['avg_fps'] for m in models]
fps_stds = [summary[m]['aggregate_metrics']['std_fps'] for m in models]

axes[0, 0].bar(models, fps_values, yerr=fps_stds, capsize=5, alpha=0.7, color=['#1f77b4', '#ff7f0e'])
axes[0, 0].set_ylabel('FPS')
axes[0, 0].set_title('Average FPS')
axes[0, 0].grid(axis='y', alpha=0.3)

# 2. Memory Usage
mem_values = [summary[m]['aggregate_metrics']['avg_memory_gb'] for m in models]
axes[0, 1].bar(models, mem_values, alpha=0.7, color=['#1f77b4', '#ff7f0e'])
axes[0, 1].set_ylabel('Memory (GB)')
axes[0, 1].set_title('Average Memory Usage')
axes[0, 1].grid(axis='y', alpha=0.3)

# 3. Detections per Frame
det_values = [summary[m]['aggregate_metrics']['avg_detections_per_frame'] for m in models]
axes[1, 0].bar(models, det_values, alpha=0.7, color=['#1f77b4', '#ff7f0e'])
axes[1, 0].set_ylabel('Detections')
axes[1, 0].set_title('Average Detections per Frame')
axes[1, 0].grid(axis='y', alpha=0.3)

# 4. Average Confidence
conf_values = [summary[m]['aggregate_metrics']['avg_confidence'] for m in models]
axes[1, 1].bar(models, conf_values, alpha=0.7, color=['#1f77b4', '#ff7f0e'])
axes[1, 1].set_ylabel('Confidence Score')
axes[1, 1].set_title('Average Confidence Score')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()

# Save plot
plot_path = comparator.results_dir / 'comparison_plots.png'
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
print(f"Comparison plot saved to: {plot_path}")

plt.show()


## Step 8: Export Results Summary


In [ ]:
# Create a comprehensive summary report
report = f"""
# MMDetection3D Model Comparison Report

## Models Compared
1. **PointPillars (KITTI-3D-car)**: Car detection only
2. **PointPillars (KITTI-3D-3class)**: Car, Pedestrian, Cyclist detection

## Datasets
- KITTI sample frames: {len(data_files)} frames

## Results Summary
"""

for model_name, model_data in summary.items():
    metrics = model_data['aggregate_metrics']
    report += f"""
### {model_name}
- **Average FPS**: {metrics['avg_fps']:.2f} ± {metrics['std_fps']:.2f}
- **Inference Time**: {metrics['avg_inference_time']:.3f}s ± {metrics['std_inference_time']:.3f}s
- **Memory Usage**: {metrics['avg_memory_gb']:.2f} GB (max: {metrics['max_memory_gb']:.2f} GB)
- **Total Detections**: {metrics['total_detections']}
- **Detections per Frame**: {metrics['avg_detections_per_frame']:.1f}
- **Average Confidence**: {metrics['avg_confidence']:.3f}

"""

report += f"""
## Output Files
All results saved to: `{RESULTS_DIR}`

- `comparison_results.json`: Complete results with all metrics
- `comparison_plots.png`: Visualization of comparison metrics
- `pointpillars_car/`: Results for car-only model
- `pointpillars_3class/`: Results for 3-class model
  - Each frame contains:
    - `*_points.ply`: Point cloud
    - `*_pred_bboxes.ply`: Predicted bounding boxes
    - `*_metadata.json`: Frame metadata and metrics

## Next Steps
1. Download results from Google Drive
2. View PLY files using `open3d_view_saved_ply.py` on your PC
3. Analyze JSON files for detailed metrics
"""

# Save report
report_path = comparator.results_dir / 'COMPARISON_REPORT.md'
with open(report_path, 'w') as f:
    f.write(report)

print(report)
print(f"\nReport saved to: {report_path}")


In [ ]:
# List all saved files
print("\n" + "="*60)
print("ALL SAVED FILES")
print("="*60)

def list_files(directory, prefix=""):
    """Recursively list all files."""
    files = []
    for item in Path(directory).iterdir():
        if item.is_file():
            files.append(str(item.relative_to(Path(RESULTS_DIR))))
        elif item.is_dir():
            files.extend(list_files(item, prefix + "  "))
    return files

all_files = list_files(RESULTS_DIR)
for f in sorted(all_files):
    print(f"  {f}")

print(f"\nTotal files: {len(all_files)}")
print(f"Results location: {RESULTS_DIR}")


## Summary

✅ **Completed:**
- Compared 2 models (PointPillars car vs 3-class)
- Processed sample KITTI data
- Saved `.png` visualizations (via MMDet3D's built-in)
- Saved `.ply` point clouds with predictions
- Saved `.json` metadata with all metrics
- Computed comparison metrics (FPS, memory, detections, confidence)
- Generated comparison plots

📁 **Download Results:**
All files are saved to Google Drive. Download the `mmdet3d_comparison` folder to view on your PC.

🔍 **View 3D Results:**
Use `open3d_view_saved_ply.py` on your PC to view the PLY files:
```bash
python open3d_view_saved_ply.py --dir /path/to/mmdet3d_comparison/pointpillars_car --basename 000008
```
